In [2]:
import os
from pathlib import Path
import sys

In [3]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    model_path = 'D:\Dataset\{0}\models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    model_path = '/Volumes/Dataset/{0}/models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    src_path = Path(project_path, 'src')

utils_path = str(Path(project_path, 'utils'))
# including the project folder and the utils folder
if utils_path not in ''.join(sys.path):
    sys.path.extend([str(project_path), utils_path, str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/clpsych
data path = /home/guerramarj/github/clpsych/dataset
model path = /home/guerramarj/github/clpsych/models
sys.path = ['/cm/local/apps/cuda/libs/current/pynvml', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/lib-dynload', '', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6/site-packages/IPython/extensions', '/home/guerramarj/.ipython', '/home/guerramarj/github/clpsych', '/home/guerramarj/github/clpsych/utils', '/home/guerramarj/github/clpsych/src']


In [4]:
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from utils.datapath import data_path_scripts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import precision_recall_fscore_support, classification_report
from numpy import interp 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import re
import itertools
from collections import Counter
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Dropout
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Adam
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras import backend
from keras import metrics
import tensorflow as tf
import _pickle as pickle 
import stanfordnlp
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.datasets import imdb 
from utils.data_helpers import load_data_and_labels
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import multi_gpu_model
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 13]

# seed for numpy and sklearn
random_state = 7
np.random.seed(random_state)

Using TensorFlow backend.


In [5]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
assert 'GPU' in str(device_lib.list_local_devices())
print('Tensorflow recognizes GPUs')

# confirm Keras sees the GPU
from keras import backend
available_gpu = backend.tensorflow_backend._get_available_gpus()
assert len(available_gpu) > 0
available_gpus = len(available_gpu)
print('number of available GPUs = {0}'.format(available_gpus))
print('list of GPUs = {0}\n'.format(available_gpu))

Tensorflow recognizes GPUs
number of available GPUs = 4
list of GPUs = ['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3']



In [6]:
def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred-cnt'] = pred_cnt
    class_report_df['pred-cnt'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [17]:
def f1_score_custom(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

In [8]:
def auc_metric(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    backend.get_session().run(tf.local_variables_initializer())
    return auc

In [9]:
%ls ../dataset

clpsych19_public_crowd_train.csv         risk_tbs.csv*
clpsych19_public_shared_task_posts.csv   risk_title_body.csv
clpsych19_public_task_B_train_posts.csv  sentiment_per_post.csv
database.sqlite                          sentiment_per_user.csv
post_risklabel.csv*                      sentiment_user.csv
post_sent_user_risk.csv                  sent_user.csv
post_user_risk                           Tweets.csv
post_user_risk.csv                       twitter-airline-sentiment.zip
preprocessed_corpus.pkl                  users.csv


In [10]:
data = pd.read_csv(Path(data_path, 'sentiment_per_post.csv'))
data.head()

,post_id,sent_1,sent_2,sent_3,sent_4,sent_5,user_id,risk_label
0,sodud,0.093305,0.389148,0.250383,0.195445,0.071718,40130,a
1,soegj,0.108678,0.342917,0.277294,0.197759,0.073352,40130,a
2,t1efg,0.044401,0.322564,0.545451,0.072668,0.014916,40130,a
3,tprco,0.034358,0.335214,0.550532,0.070563,0.009332,40130,a
4,ttak5,0.055131,0.399358,0.453258,0.077912,0.014341,40130,a


In [11]:
data.dropna(subset=['risk_label'], inplace=True)
data.head()

,post_id,sent_1,sent_2,sent_3,sent_4,sent_5,user_id,risk_label
0,sodud,0.093305,0.389148,0.250383,0.195445,0.071718,40130,a
1,soegj,0.108678,0.342917,0.277294,0.197759,0.073352,40130,a
2,t1efg,0.044401,0.322564,0.545451,0.072668,0.014916,40130,a
3,tprco,0.034358,0.335214,0.550532,0.070563,0.009332,40130,a
4,ttak5,0.055131,0.399358,0.453258,0.077912,0.014341,40130,a


In [12]:
lb_make = LabelEncoder()

y = data[['risk_label']].copy()
y.loc[:, 'risk_label'] = lb_make.fit_transform(y['risk_label'])
y_temp = pd.get_dummies(data['risk_label'].astype('category')).copy()
y[y_temp.keys()] = y_temp
x = data.drop('risk_label', axis=1).copy()

In [18]:
cvscores = list()
auc_scores = list()
report_with_auc_list = list()

# define 10-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=7)

print('5 fold CV starting')
for cv_ix, x_y in enumerate(kfold.split(x, y)):
    
    X_train = x.loc[x_y[0]]
    y_train = y.loc[x_y[0]]
    
    X_test = x.loc[x_y[1]]
    y_test = y.loc[x_y[1]]

    x_train = X_train.drop(['post_id', 'user_id'], axis=1).copy()
    x_test = X_test.drop(['post_id', 'user_id'], axis=1).copy()
    y_train_risk = y_train['risk_label'].copy()
    y_test_risk = y_test['risk_label'].copy()
    y_train.drop('risk_label', axis=1, inplace=True)
    y_test.drop('risk_label', axis=1, inplace=True)

    # basic neural network
    model = Sequential()
    model.add(Dense(25, input_dim=np.shape(x_train)[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))

    model = multi_gpu_model(model, gpus=available_gpus)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1_score_custom])
    model.fit(x_train, y_train, epochs=10, batch_size=100, verbose=1)

    y_pred_train = np.argmax(model.predict(x_train), axis=1)
    y_pred_test = np.argmax(model.predict(x_test), axis=1)

    report_with_auc = class_report(
        y_true=y_test_risk, 
        y_pred=y_pred_test, 
        y_score=model.predict(x_test),
        average='macro')
    
    cv_column = [cv_ix]
    cv_column.extend( [''] * (report_with_auc.index.shape[0] - 1))
    report_with_auc['Fold'] = cv_column
    report_with_auc['Risk-Factor'] = report_with_auc.index
    report_with_auc.set_index(['Fold', 'Risk-Factor'])
    report_with_auc_list.append(report_with_auc)

5 fold CV starting


TypeError: object of type 'Tensor' has no len()